In [1]:
from recovar import RepresentationLearningSingleAutoencoder, RepresentationLearningDenoisingSingleAutoencoder, RepresentationLearningMultipleAutoencoder
from kfold_trainer import KfoldTrainer
from config import KFOLD_SPLITS

2025-07-01 18:19:34.536055: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-01 18:19:34.554474: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-01 18:19:34.554497: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-01 18:19:34.554508: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-01 18:19:34.557968: I tensorflow/core/platform/cpu_feature_g

In [ ]:
# Experiment name.
EXP_NAME = "exp_test"

# Should be one of the RepresentationLearningSingleAutoencoder, RepresentationLearningDenoisingSingleAutoencoder, RepresentationLearningMultipleAutoencoder
MODEL_CLASSES = [RepresentationLearningSingleAutoencoder]#, RepresentationLearningDenoisingSingleAutoencoder, RepresentationLearningMultipleAutoencoder]

# Should be stead or instance.
DATASETS = ["stead"]

# Number of epochs
NUM_EPOCHS = 1

In [ ]:
# For all splits, train the model over defined datasets.
for train_dataset in DATASETS:
    for model_class in MODEL_CLASSES:
        for split in range(KFOLD_SPLITS):
            kfold_trainer = KfoldTrainer(
                EXP_NAME, model_class, train_dataset, split, epochs=NUM_EPOCHS
            )
            kfold_trainer.train()